# Import the necessary libraries:


In [6]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import roc_auc_score

# Load the training and test data :

In [7]:
# Loading training datasets
train1 = pd.read_csv('train_1.csv')
train2 = pd.read_csv('train_2_1.csv')
train3 = pd.read_csv('train_2_2.csv')

# Loading test datasets
test1 = pd.read_csv('test_1.csv')
test2 = pd.read_csv('test_2_1.csv')
test3 = pd.read_csv('test_2_2.csv')


C:\Users\Tirtha Ghosh\AppData\Local\Temp\ipykernel_21696\3609402953.py:2: DtypeWarning: Columns (126,128,143) have mixed types. Specify dtype option on import or set low_memory=False.
  train1 = pd.read_csv('train_1.csv')
C:\Users\Tirtha Ghosh\AppData\Local\Temp\ipykernel_21696\3609402953.py:3: DtypeWarning: Columns (675,676,677) have mixed types. Specify dtype option on import or set low_memory=False.
  train2 = pd.read_csv('train_2_1.csv')
C:\Users\Tirtha Ghosh\AppData\Local\Temp\ipykernel_21696\3609402953.py:4: DtypeWarning: Columns (675,676,677) have mixed types. Specify dtype option on import or set low_memory=False.
  train3 = pd.read_csv('train_2_2.csv')


In [8]:
# Check the shape of the training and test datasets
print('Train1.shape------->', train1.shape)
print('Train2_1.shape----->', train2.shape)
print('Train2_2.shape----->', train3.shape)

print('Test1.shape------->', test1.shape)
print('Test2_1.shape----->', test2.shape)
print('Test2_2.shape----->', test3.shape)


Train1.shape-------> (100000, 168)
Train2_1.shape-----> (90300, 678)
Train2_2.shape-----> (47976, 678)
Test1.shape-------> (100000, 167)
Test2_1.shape-----> (81300, 678)
Test2_2.shape-----> (48085, 678)


In [9]:
# Select the categorical columns from the train1 dataset
categorical_columns = train1.select_dtypes(include=['object', 'category']).columns

# Select the categorical columns from the test1 dataset
test_categorical_columns = test1.select_dtypes(include=['object', 'category']).columns


In [10]:
# Drop the categorical columns from the train1 dataset
df = train1.drop(columns=categorical_columns)

# Drop the categorical columns from the test1 dataset
test_df = test1.drop(columns=test_categorical_columns)

In [11]:
# Checking the number of null values in each column of the train set
null_values = df.isnull().sum()
print(null_values)

label          0
col_1          0
col_2          0
col_3          0
col_4          0
           ...  
col_159     1087
col_160     1087
col_162        0
col_163    50277
col_164     1096
Length: 152, dtype: int64


## Identify features that are highly correlated with the target column:


In [14]:
# Assuming df is your DataFrame and 'target' is the column you are trying to predict
correlation_matrix = df.corr()

# Get the correlations with the target variable
correlation_with_target = correlation_matrix['label'].sort_values(ascending=False)

# Select the top features based on their absolute correlation values
f = correlation_with_target.index[1:100]  # Select top 10 features excluding the target itself

print("Top Features Based on Correlation:", f)


Top Features Based on Correlation: Index(['col_128', 'col_24', 'col_25', 'col_104', 'col_68', 'col_54', 'col_122',
       'col_69', 'col_50', 'col_48', 'col_26', 'col_130', 'col_52', 'col_2',
       'col_147', 'col_115', 'col_55', 'col_53', 'col_49', 'col_46', 'col_27',
       'col_3', 'col_16', 'col_132', 'col_45', 'col_127', 'col_47', 'col_66',
       'col_60', 'col_129', 'col_56', 'col_57', 'col_63', 'col_67', 'col_28',
       'col_77', 'col_29', 'col_76', 'col_155', 'col_131', 'col_71', 'col_74',
       'col_75', 'col_51', 'col_133', 'col_73', 'col_23', 'col_134', 'col_11',
       'col_4', 'col_18', 'col_136', 'col_105', 'col_135', 'col_58', 'col_138',
       'col_106', 'col_72', 'col_109', 'col_137', 'col_159', 'col_139',
       'col_107', 'col_142', 'col_110', 'col_78', 'col_19', 'col_70', 'col_89',
       'col_91', 'col_93', 'col_22', 'col_20', 'col_163', 'col_124', 'col_65',
       'col_126', 'col_87', 'col_112', 'col_120', 'col_101', 'col_119',
       'col_88', 'col_96', 'col_

In [12]:
f=[ 'col_24', 'col_25', 'col_68', 'col_54', 'col_122',
       'col_69', 'col_50', 'col_48', 'col_26',  'col_2',
       'col_147', 'col_127', 'col_66',
       'col_60', 'col_63', 'col_67',  'col_76',  'col_74',
       'col_75',  'col_73',  'col_58',  'col_72', 'col_142',  'col_70', 'col_89',
       'col_91', 'col_93', 'col_22', 'col_163', 'col_124', 
       'col_126', 'col_119',
        'col_83', 'col_85', 'col_99',
       'col_43', 'col_14']

In [13]:
# split the dataset in to the X and Y column :
X=df[f]
Y=df['label']

# Impute the missing values (NaNs) on train data :


In [14]:
from sklearn.impute import KNNImputer

# Initialize the KNN Imputer with k=5 neighbors
knn_imputer = KNNImputer(n_neighbors=5)

# Fit and transform the datab
impute_df= pd.DataFrame(knn_imputer.fit_transform(X), columns=X.columns)

In [16]:
df123=impute_df
df123.shape

(100000, 37)

### Identify the range of features that provide the highest cross-validation score:


In [ ]:
for i in range(40,100):
    l=f[:i]
    
    X=df[l]
    Y=df['label']
    
    # Example: Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    
    # Initialize RandomForestClassifier
    rf_clf = RandomForestClassifier(n_estimators =824,min_samples_split=10,min_samples_leaf=1,max_features='sqrt',
                                    max_depth=20,bootstrap=False)
    #'n_estimators': 824, 'max_depth': 20, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'bootstrap': False
    # Output cross-validation scores
    cv_scores = cross_val_score(rf_clf, X_train, y_train, cv=5, scoring='roc_auc', n_jobs=-1)
    
    print(i,'-------->',"Mean CV AUC-ROC Score:", cv_scores.mean())

## Hyperparameter tuning of RandomForest model using the optuna:

In [31]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# Ensure 'X' and 'Y' are your features and target variable
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

def objective(trial):
    # Define hyperparameter search space
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_categorical('max_features', [None, 'sqrt', 'log2'])
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])

    # Define the Random Forest model with suggested hyperparameters
    rf_clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,  # Updated to avoid 'auto'
        bootstrap=bootstrap,
        random_state=42
    )

    # Train the model
    rf_clf.fit(X_train, y_train)

    # Predict probabilities on the test set and calculate AUC-ROC score
    score = roc_auc_score(y_test, rf_clf.predict_proba(X_test)[:, 1])
    return score

# Create a study object and optimize it
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)

# Print the best hyperparameters
print("Best Hyperparameters:", study.best_params)


The Optuna code took around 2 hours to execute and identify the best features. Since I ran the code in a separate Python file, the execution result is not displayed here. However, the optimal hyperparameters for my model are as follows: ('
n_estimator's: 824,' min_samples_spli': 10,' min_samples_lea': 1,' max_feature's: 'sqrt',' max_dept'h: 20,' bootstra'p: False).


## Train the model after hyper parameter tuning :

Out of the 100 best features I extracted, the first 83 features yielded the highest cross-validation score.


In [37]:
l=f[:83]
    
X1=df123[l]
Y=df['label']
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, train_test_split

    # Example: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X1, Y, test_size=0.2, random_state=42)
    
    # Initialize RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators =824,min_samples_split=10,min_samples_leaf=1,max_features='sqrt',
                                    max_depth=20,bootstrap=False)
    #'n_estimators': 824, 'max_depth': 20, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'bootstrap': False
    # Output cross-validation scores
cv_scores = cross_val_score(rf_clf, X_train, y_train, cv=5, scoring='roc_auc', n_jobs=-1)
    
print('-------->',"Mean CV AUC-ROC Score:", cv_scores.mean())

--------> Mean CV AUC-ROC Score: 0.8178833587994057


## Extract the features from the test data that were used to train the model:


In [ ]:
test_df1=test1[l]

# Impute the missing values (NaNs) on test data :

In [ ]:
from sklearn.impute import KNNImputer

# Initialize the KNN Imputer with k=5 neighbors
knn_imputer = KNNImputer(n_neighbors=5)

# Fit and transform the datab
test_X1 = pd.DataFrame(knn_imputer.fit_transform(test_df1), columns=test_df1.columns)

In [ ]:
# Save the test data as a CSV file on my system for future reference:
test_X1.to_csv('last_df.csv',index=False)

In [50]:
test=pd.read_csv('last_df.csv')

In [51]:
test.shape

(100000, 99)

## Execute the model on the test data and save the predicted values to a CSV file:


In [ ]:
# Import required libraries
l=f[:83]
    
X=df[l]
Y=df['label']
test1=test1[X.columns]
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Assuming X is your feature set and Y is your target (label)
# Split the dataset into training and testing sets (if not done already)
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest Classifier with valid parameters
model= RandomForestClassifier(n_estimators =824,min_samples_split=10,min_samples_leaf=1,max_features='sqrt',
                                    max_depth=20,bootstrap=False)

# Train the model on the training data
rf_clf.fit(X, Y)

# Load your new test data into a DataFrame (e.g., test_new)
# Ensure it has the same columns as X
# For example, test_new = pd.read_csv('new_test_data.csv')

# Make predictions on the new test data
predictions = rf_clf.predict_proba(test1)[:, 1]  # Predict probabilities for the positive class

# Convert predictions to a list if needed
predictions = list(predictions)

# If you want to save the predictions to a CSV file
test = pd.read_csv('test.csv')

sub = test[['loan_id']].copy()
sub['prob'] = predictions

# Save to CSV
sub.to_csv('RFC81.csv', index=False)


In [17]:
l=f[:83]
    
X1=df123[l]
Y=df['label']
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, train_test_split

    # Example: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X1, Y, test_size=0.2, random_state=42)
    
    # Initialize RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators =824,min_samples_split=10,min_samples_leaf=1,max_features='sqrt',
                                    max_depth=20,bootstrap=False)
    #'n_estimators': 824, 'max_depth': 20, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'bootstrap': False
    # Output cross-validation scores
cv_scores = cross_val_score(rf_clf, X_train, y_train, cv=5, scoring='roc_auc', n_jobs=-1)
    
print('-------->',"Mean CV AUC-ROC Score:", cv_scores.mean())

--------> Mean CV AUC-ROC Score: 0.8145764376037954


In [ ]:
|